# Transforming the Meta-DataFrame into a Neo4j Event Log Class Diagram Graph
The following script demonstrates how the Meta-DataFrame can be modelled in Neo4j as an Event Log Class Diagram Graph of a given event log.

In [65]:
# import the libraries
import json
import pandas as pd
import numpy
import re #regular expressions for working with text data, especially extracting information from a piece of text

In [66]:
# Change the title names to circumvent certain Neo4j whitespace issues. The index_to_neo4j dictionary  maps keys will map the values from the Meta-DataFrame to the Neo4j graph later
index_to_neo4j = {
    'Title': 'title',
    'Data_type': 'data_type',
    'Number_of_entries': 'num_entries',
    'Number_of_unique_entries': 'num_unique_entries',
    'Number_of_duplicate_entries':'num_duplicate_entries',
    'Number_of_undefined_entries': 'num_undefined_entries',
    'Percentage_of_undefined_entries': 'percentage_undefined_entries',
}
index_to_neo4j

{'Title': 'title',
 'Data_type': 'data_type',
 'Number_of_entries': 'num_entries',
 'Number_of_unique_entries': 'num_unique_entries',
 'Number_of_duplicate_entries': 'num_duplicate_entries',
 'Number_of_undefined_entries': 'num_undefined_entries',
 'Percentage_of_undefined_entries': 'percentage_undefined_entries'}

In [67]:
#Define the Meta-DataFrame and generate the first command to clear any related event data in Neo4j
event_log_name = 'BPIC_Synthetic_Split_3' #change accordingly for each EL
print('// Delete all nodes and their relationships:')
print(f'MATCH (node:{event_log_name}) DETACH DELETE node;')
print()

// Delete all nodes and their relationships:
MATCH (node:BPIC_Synthetic_Split_3) DETACH DELETE node;



In [68]:
#Import the event log Meta-DataFrame based on event_log_name above
overview_table = pd.read_csv(f'analysis/overview_{event_log_name}.csv', index_col = 'Property') #use Property as the index_column
overview_table = overview_table.T #switch table layout
overview_table

Property,caseID,event time:timestamp,event concept:name,eventID,case Sub spend area text,case Item Category,case Name,case Goods Receipt,event org:resource
Unnamed: 0,0,1,2,3,4,5,6,7,8
Supplier2_Comparative_Mining,Supplier2_Comparative_Mining,Supplier2_Comparative_Mining,Supplier2_Comparative_Mining,Supplier2_Comparative_Mining,Supplier2_Comparative_Mining,Supplier2_Comparative_Mining,Supplier2_Comparative_Mining,Supplier2_Comparative_Mining,Supplier2_Comparative_Mining
Title,Case ID,Timestamp,Activity,Event ID,case Sub spend area text,case Item Category,case Name,case Goods Receipt,event org:resource
Importance,Mandatory attribute,Mandatory attribute,Mandatory attribute,Mandatory attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute,Additional attribute
Level,Case level,Event level,Event level,Event level,Case level,Case level,Case level,Case level,Event level
Data type,string,datetime64[ns],string,float64,string,string,string,bool,string
Number of entries,349527,349527,349527,349527,349527,349527,349527,349527,349527
Number of unique entries,70874,42245,27,362,121,4,1262,2,425
List up to 100 unique entries to view the structure of the event attributes,"['4507032138 1', '4507032139 1', '4507032140 1...",[numpy.datetime64('2018-05-28T22:53:00.0000000...,"['Record Service Entry Sheet', 'Record Goods R...","[469000000000000.0, 470000000000000.0, 4710000...","['Other Logistics Services', 'Road Packed', 'S...","['3-way match, invoice after GR', 'Consignment...","['vendor_0225', 'vendor_0226', 'vendor_0228', ...","[True, False]","['NONE', 'batch_06', 'user_013', 'user_002', '..."
Length of the list of up to 100 unique entries,100,100,27,100,100,4,100,2,100


In [69]:
# Gather overview_columns
overview_columns = overview_table.columns

#View
overview_columns


Index(['caseID', 'event time:timestamp', 'event concept:name', 'eventID ',
       'case Sub spend area text', 'case Item Category', 'case Name',
       'case Goods Receipt', 'event org:resource'],
      dtype='object', name='Property')

<h1>Creating the Neo4j Commands</h1>
The author hardcoded the Neo4j commands as changes could be made easily - when adjustments have to be made to the Neo4jDB instance.

In [70]:
# create a two lists
neo4j_lines = []
mandatory_columns = []

for overview_column in overview_columns:
  spalte = overview_table[overview_column]
  spalte_json_string = spalte.to_json(default_handler=str)
  neo4j_properties = json.loads(spalte_json_string)

  overview_level = neo4j_properties['Level']

  property_label_identifier = neo4j_properties['Importance']
  if property_label_identifier == 'Mandatory attribute':
    mandatory_columns.append(overview_column)

  properties_list_of_strings = [f'`{key}`: "{value}"' for (key, value) in neo4j_properties.items()]

  neo4j_create_command = ''
  neo4j_create_command += f'''CREATE (`{overview_column}`:`{overview_column}`:`{overview_level}`:`{property_label_identifier}`:{event_log_name} '''
  neo4j_create_command += f'''{{name: '{overview_column}', {', '.join(properties_list_of_strings)}}})'''

  neo4j_lines.append(neo4j_create_command)

neo4j_lines.append('CREATE (`caseID`) - [:CASE_TO_EVENT] -> (`eventID `)')
neo4j_lines.append('CREATE (`eventID `) - [:EVENT_RELATIONSHIP] -> (`event concept:name`)')
neo4j_lines.append('CREATE (`eventID `) - [:EVENT_RELATIONSHIP] -> (`event time:timestamp`)')

for overview_column in overview_columns:
  if overview_column not in mandatory_columns and 'event' in overview_column:
    neo4j_create_command = f'''CREATE (`eventID `) - [:EVENT_RELATIONSHIP] -> (`{overview_column}`)'''
    neo4j_lines.append(neo4j_create_command)
  if overview_column not in mandatory_columns and 'case' in overview_column:
    neo4j_create_command = f'''CREATE (`caseID`) - [:CASE_RELATIONSHIP] -> (`{overview_column}`)'''
    neo4j_lines.append(neo4j_create_command)

# To add the preceding sequence of Cypher CREATE commands.
neo4j_lines.append(';')

neo4j_command = '\n'.join(neo4j_lines)

neo4j_command = '// Create all nodes and their relationships\n' + neo4j_command

with open(f'analysis/overview_{event_log_name}.cypher', 'w') as overview_file:
  print(neo4j_command, file=overview_file)

In [71]:
print('// Show all nodes and their relationships:')
print(f'MATCH (event_log:{event_log_name}) RETURN event_log;')
print()

// Show all nodes and their relationships:
MATCH (event_log:BPIC_Synthetic_Split_3) RETURN event_log;



In [72]:
# The case table is an exploratory acitivty to model the longest case to visualize all event contained in that case in Neo4j commands

case_table = pd.read_csv(f'analysis/longest_case_in_{event_log_name}.csv', index_col='EventIndex')
case_table


# The case table is an event table that only contains events belonging to a certain case.
# Index(['caseID', 'event time:timestamp', 'event concept:name', 'eventID ','case Spend area text', 'case Document Type','case Sub spend area text', 'case Purch. Doc. Category name','case Vendor', 'case Item Type', 'case Item Category', 'case Spend classification text', 'case Source', 'case Name', 'case GR-Based Inv. Verif.', 'case Item', 'case concept:name','case Goods Receipt', 'event User', 'event org:resource','event Cumulative net worth (EUR)'],dtype='object', name='Property')

cypher_code = 'CREATE\n'
for event_index in case_table.index:
  event = case_table.loc[event_index]
  cypher_code += \
    f'''  (event_{event_index}:`{event['event concept:name']}` {{\n''' +\
    f'''    `caseID`: "{event['caseID']}",\n''' +\
    f'''    `event time:timestamp`: "{event['event time:timestamp']}",\n''' +\
    f'''    `event concept:name`: "{event['event concept:name']}",\n''' +\
    f'''    `eventID `: "{event['eventID ']}",\n''' +\
    f'''    `case Sub spend area text`: "{event['case Sub spend area text']}",\n''' +\
    f'''    `case Item Category`: "{event['case Item Category']}",\n''' +\
    f'''    `case Name`: "{event['case Name']}",\n''' +\
    f'''    `case Goods Receipt`: "{event['case Goods Receipt']}",\n''' +\
    f'''    `event org:resource`: "{event['event org:resource']}",\n''' +\
    f'''  }}),\n'''

for first_event_index, second_event_index in zip(case_table.index, case_table.index[1:]):
  first_event = case_table.loc[first_event_index]
  second_event = case_table.loc[second_event_index]
  cypher_code += f'''  (event_{first_event_index})-[:DIRECTLY_FOLLOWS]->(event_{second_event_index}),\n'''

# Delete the last comma symbol occurrence.
cypher_code = cypher_code[:-2]
cypher_code += '\n;'

with open(f'analysis/longest_case_in_{event_log_name}.cypher', 'w') as case_file:
  print(cypher_code, file=case_file)


# End of Script 2 Transforming the Meta-DataFrame into a Neo4j Event Log Class Diagram Graph
Author: Kyle Smith <br>
Script: For Masterthesis <br>
University of Camerino & University of Applied Sciences Northwestern Switzerland